In [2]:
from data import HomDensityDataset

/cis/home/yma93/miniconda3/envs/ml/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
data = HomDensityDataset("data/", N=1000, n=80, d=1, graph_model="Sociality", task = "4-cycle")
data[0].x

Processing...
Done!


tensor([[[ 0.3569],
         [-0.4193],
         [-0.7296],
         [-0.4778],
         [-0.9291],
         [ 1.4990],
         [ 1.1816],
         [-0.8419],
         [ 1.6745],
         [ 1.2441],
         [ 0.4521],
         [ 1.3654],
         [-0.8265],
         [-1.2282],
         [ 0.5207],
         [-1.5179],
         [ 1.2648],
         [ 1.5140],
         [ 0.5479],
         [-0.5378],
         [-1.0946],
         [-1.0520],
         [-0.5495],
         [-1.1606],
         [ 0.9384],
         [-0.6212],
         [ 0.3688],
         [ 1.3401],
         [-2.0136],
         [-2.0552],
         [ 0.8003],
         [-0.4805],
         [ 0.5222],
         [ 0.5198],
         [-1.6651],
         [ 1.0681],
         [-1.2428],
         [ 0.5405],
         [ 0.4542],
         [ 0.1899],
         [-1.6580],
         [-0.2362],
         [-0.7400],
         [ 2.3431],
         [-0.0880],
         [-0.1582],
         [ 1.4055],
         [-0.6246],
         [ 1.1098],
         [-0.0812],


In [1]:
import torch
from torch_geometric.datasets import Amazon, SNAPDataset
from sklearn.cluster import spectral_clustering
import torch_geometric.utils as pyg_utils
import matplotlib.pyplot as plt
from torch_geometric.data import NeighborSampler

# Load the dataset
dataset = SNAPDataset(root="data", name="soc-ca-astroph")

/cis/home/yma93/miniconda3/envs/ml/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
len(dataset)

1

In [3]:
data = dataset[0]

In [4]:
N = 1000
chosen_nodes = torch.arange(N)
mask = (data.edge_index[0] < N) | (data.edge_index[1] < N)
filtered_edge_index = data.edge_index[:, mask]

In [5]:
len(filtered_edge_index.unique())

2657

In [7]:
A = pyg_utils.to_dense_adj(filtered_edge_index)[0]
degree = A.sum(dim=0)
print(degree.min(), degree.max(), degree.mean())

tensor(0.) tensor(137.) tensor(0.0474)


In [16]:
degree = A.sum(dim=0)
sorted_indices = degree.argsort()
A_sorted = A[sorted_indices, : ][:, sorted_indices]

: 

In [9]:
plt.imshow(A)

: 